In [1]:
import os
import pandas as pd
from bs4 import BeautifulSoup
import sys
import re
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
ps = PorterStemmer()

In [4]:
print(os.getcwd());

/Users/jansovka/Desktop/AA/2017-09-12_Text_mining


In [ ]:
# if necessary change the directory
#os.chdir('c:\\Users\..')

In [6]:
data = pd.read_csv("nightlife_sanfrancisco_en.csv", header=0, delimiter=",", encoding="windows-1250")

In [7]:
# iexplore data set
data.shape

(34086, 3)

In [8]:
data.columns.values

array(['company', 'text', 'stars'], dtype=object)

In [10]:
print(data["text"][0])

Great experience with Joey for a family karaoke night! He had every song that was requested by attendees. He literally had over 50,000+ songs to choose from -- everything from top 40s, R&B, hip hop, country, pop, and every song from any and every decade. He also played great dance music during transitions between singers.  

He was extremely professional and accommodating throughout the entire planning process and during the actual event. We will definitely be using Joey again for future events - I highly recommend him for any of his services!



In [11]:
# Remove stop words from "words"
import nltk      # import stop words
nltk.download('popular')  # Download text data sets, including stop words

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/jansovka/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/jansovka/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/jansovka/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/jansovka/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/jansovka/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/jansovka/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downlo

True

In [12]:
from nltk.corpus import stopwords # Import the stop word list
print(stopwords.words("english"))
#words = [w for w in words if not w in stopwords.words("english")]
#print words #  "u" before each word indicates that Python is internally representing each word as a unicode string

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

In [14]:
# Clean all records
def text_to_words( raw_text ):
    # 1. Remove end of line
    without_end_line = re.sub('\n', ' ', raw_text)
    # 2. Remove start of line
    without_start_line = re.sub('\r', ' ', without_end_line)
    # 3. Remove punctuation
    without_punctual = re.sub(u'[\W_]+',' ',without_start_line )
    # 4. Replace number by XnumberX
    without_number = re.sub('(\d+\s*)+', ' XnumberX ', without_punctual)
    # 5. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", without_number) 
    # 6. Convert to lower case
    lower_case = letters_only.lower()
    # 7. Split into individual words
    words = lower_case.split()
    # 8. stemming  - algorithms Porter stemmer
    meaningful_words = [ps.stem(word) for word in words]
    # 9.Remove stop words 
    # Redundant step, removing later in Creating the bag of words step
    #stops = set(stopwords.words("english"))                  
    #meaningful_words = [w for w in words if not w in stops]   
    # 10. Join the words back into one string separated by space and return the result.
    return( " ".join( meaningful_words ))
    #return (meaningful_words) 

In [15]:
clean_text = text_to_words( data["text"][0] )
print(clean_text)

great experi with joey for a famili karaok night he had everi song that wa request by attende he liter had over xnumberx song to choos from everyth from top xnumberx s r b hip hop countri pop and everi song from ani and everi decad he also play great danc music dure transit between singer he wa extrem profession and accommod throughout the entir plan process and dure the actual event we will definit be use joey again for futur event i highli recommend him for ani of hi servic


In [18]:
# Get the number of text based on the dataframe column size
num_text = data["text"].size
# Initialize an empty list to hold the clean text
clean_data = []
# Loop over each text; create an index i that goes from 0 to the length
print("Cleaning and parsing the data set text...\n")
clean_data = []
for i in range( 0, num_text ):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print("Text %d of %d\n" % ( i+1, num_text ))                                                                    
    clean_data.append( text_to_words( data["text"][i] )) # in case of error run "pip install -U nltk"

Cleaning and parsing the data set text...

Text 1000 of 34086

Text 2000 of 34086

Text 3000 of 34086

Text 4000 of 34086

Text 5000 of 34086

Text 6000 of 34086

Text 7000 of 34086

Text 8000 of 34086

Text 9000 of 34086

Text 10000 of 34086

Text 11000 of 34086

Text 12000 of 34086

Text 13000 of 34086

Text 14000 of 34086

Text 15000 of 34086

Text 16000 of 34086

Text 17000 of 34086

Text 18000 of 34086

Text 19000 of 34086

Text 20000 of 34086

Text 21000 of 34086

Text 22000 of 34086

Text 23000 of 34086

Text 24000 of 34086

Text 25000 of 34086

Text 26000 of 34086

Text 27000 of 34086

Text 28000 of 34086

Text 29000 of 34086

Text 30000 of 34086

Text 31000 of 34086

Text 32000 of 34086

Text 33000 of 34086

Text 34000 of 34086



In [19]:
# Compare original and edited text
data['text'][0]

'Great experience with Joey for a family karaoke night! He had every song that was requested by attendees. He literally had over 50,000+ songs to choose from -- everything from top 40s, R&B, hip hop, country, pop, and every song from any and every decade. He also played great dance music during transitions between singers.  \r\n\r\nHe was extremely professional and accommodating throughout the entire planning process and during the actual event. We will definitely be using Joey again for future events - I highly recommend him for any of his services!\r\n'

In [20]:
clean_data[0]

'great experi with joey for a famili karaok night he had everi song that wa request by attende he liter had over xnumberx song to choos from everyth from top xnumberx s r b hip hop countri pop and everi song from ani and everi decad he also play great danc music dure transit between singer he wa extrem profession and accommod throughout the entir plan process and dure the actual event we will definit be use joey again for futur event i highli recommend him for ani of hi servic'

In [21]:
print("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = 'english',   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_data)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

Creating the bag of words...



In [22]:
print(train_data_features.shape)

(34086, 5000)


In [23]:
# Take a look at the words in the vocabulary
vocab = vectorizer.get_feature_names()
print(vocab)

['abandon', 'abc', 'abil', 'abl', 'abound', 'abov', 'absinth', 'absolut', 'absorb', 'absurd', 'abund', 'abus', 'abv', 'ac', 'accent', 'accept', 'access', 'accid', 'accident', 'accommod', 'accomod', 'accompani', 'accomplish', 'accord', 'accordingli', 'account', 'accur', 'accus', 'ace', 'ach', 'achiev', 'acid', 'acknowledg', 'acoust', 'acrobat', 'act', 'action', 'activ', 'actual', 'ad', 'adam', 'add', 'addict', 'addit', 'address', 'adequ', 'adjac', 'adjust', 'admir', 'admiss', 'admit', 'admittedli', 'adobo', 'ador', 'adorn', 'adult', 'advanc', 'advantag', 'adventur', 'advertis', 'advic', 'advis', 'aesthet', 'af', 'affair', 'affect', 'afford', 'aficionado', 'aforement', 'afraid', 'african', 'afrolici', 'afternoon', 'aftertast', 'afterward', 'age', 'aggress', 'agnolotti', 'ago', 'agre', 'ah', 'ahead', 'ahem', 'ahh', 'ahhh', 'ahi', 'aid', 'aim', 'ain', 'aint', 'aioli', 'air', 'airi', 'aisl', 'aka', 'al', 'ala', 'alarm', 'albeit', 'album', 'alcohol', 'ale', 'alert', 'alex', 'alic', 'alien', 

In [24]:
import numpy as np

# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    print(count, tag)

31 abandon
25 abc
70 abil
1466 abl
22 abound
551 abov
78 absinth
1120 absolut
35 absorb
37 absurd
44 abund
32 abus
38 abv
66 ac
87 accent
277 accept
244 access
49 accid
98 accident
475 accommod
62 accomod
146 accompani
29 accomplish
106 accord
45 accordingli
107 account
52 accur
50 accus
26 ace
25 ach
39 achiev
69 acid
95 acknowledg
332 acoust
25 acrobat
705 act
180 action
134 activ
2668 actual
440 ad
53 adam
626 add
74 addict
581 addit
81 address
59 adequ
46 adjac
79 adjust
67 admir
158 admiss
243 admit
64 admittedli
20 adobo
169 ador
54 adorn
220 adult
192 advanc
109 advantag
146 adventur
148 advertis
167 advic
87 advis
80 aesthet
37 af
35 affair
33 affect
269 afford
33 aficionado
52 aforement
115 afraid
39 african
26 afrolici
513 afternoon
28 aftertast
173 afterward
592 age
141 aggress
21 agnolotti
895 ago
302 agre
98 ah
302 ahead
21 ahem
28 ahh
36 ahhh
75 ahi
34 aid
45 aim
154 ain
28 aint
177 aioli
473 air
37 airi
38 aisl
138 aka
46 al
88 ala
21 alarm
70 albeit
33 album
1372 alcoho

In [27]:
# Using in model, random forest example
print("Training the random forest...")
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, data["stars"] )

Training the random forest...
